In [ ]:
import sys
import nltk
import sklearn
import pandas as pd
import numpy
import os
import numpy as np


In [ ]:
transcript_directory = "/content/drive/MyDrive/DAIC-WOZ/" 
label_filepath = "/content/drive/MyDrive/DAIC-WOZ/train_split_Depression_AVEC2017.csv"
validation="/content/drive/MyDrive/DAIC-WOZ/dev_split_Depression_AVEC2017.csv"


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

### Reading for 1 patient: id=300

In [ ]:
eg_transcript_filepath = transcript_directory + "300_P/300_TRANSCRIPT.csv" #insert file path to transcript here 
eg_transcript_filename = os.path.basename(eg_transcript_filepath)

df_transcript = pd.read_csv(eg_transcript_filepath, sep = '\t')
df_transcript.head()

,start_time,stop_time,speaker,value
0,36.588,39.668,Ellie,hi i'm ellie thanks for coming in today
1,39.888,43.378,Ellie,i was created to talk to people in a safe and ...
2,43.728,48.498,Ellie,think of me as a friend i don't judge i can't ...
3,49.188,52.388,Ellie,i'm here to learn about people and would love ...
4,52.658,58.958,Ellie,i'll ask a few questions to get us started and...


In [ ]:
df_labels1 = pd.read_csv(label_filepath)



df_labels1["Participant_ID"][0]

303

In [ ]:
df_label2=pd.read_csv(validation)

In [ ]:
df_labels = pd.concat([df_label2, df_labels1], axis=0)


In [ ]:
df_labels

,Participant_ID,PHQ8_Binary,PHQ8_Score,Gender,PHQ8_NoInterest,PHQ8_Depressed,PHQ8_Sleep,PHQ8_Tired,PHQ8_Appetite,PHQ8_Failure,PHQ8_Concentrating,PHQ8_Moving
0,302,0,4,1,1,1,0.0,1,0,1,0,0
1,307,0,4,0,0,1,0.0,1,0,2,0,0
2,331,0,8,1,1,1,1.0,1,1,1,1,1
3,335,1,12,0,1,1,3.0,2,3,1,1,0
4,346,1,23,0,2,3,3.0,3,3,3,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...
102,485,0,2,1,0,1,0.0,0,0,0,0,1
103,486,0,4,0,1,1,0.0,1,0,1,0,0
104,487,0,0,0,0,0,0.0,0,0,0,0,0
105,488,0,0,0,0,0,0.0,0,0,0,0,0


In [ ]:
li= df_labels["Participant_ID"]

In [ ]:
str(df_labels["Participant_ID"][0])

'0    302\n0    303\nName: Participant_ID, dtype: int64'

### Retrieving text info + patient id 

In [ ]:
def get_transcript(filepath):
    filename = os.path.basename(filepath)
    participant_no = int(filename[0:3])
    
    df_transcript = pd.read_csv(filepath, sep = '\t')  
    
    df_cleaned = df_transcript.drop(['start_time', 'stop_time'], axis = 1) # drop first 2 columns
    try: 
      df_dropped_ellie = df_cleaned.set_index('speaker').drop(index = ['Ellie']) # drop anything Ellie says, she's not important
      df_cleaned = df_dropped_ellie.reset_index() 
    except KeyError:
      print("error")
    
    transcript_concat = '' 

    for index, row in df_cleaned.iterrows():
        if (index == 0):
            transcript_concat += str(row['value'])
        else:
            transcript_concat += ' ' + str(row['value'])
            
    return participant_no, transcript_concat

In [ ]:
get_transcript(transcript_directory + '300' + "_P/" + "300_TRANSCRIPT.csv")


(300,
 "good atlanta georgia um my parents are from here um i love it i like the weather i like the opportunities um yes um it took a minute somewhat easy congestion that's it um i took up business and administration uh yeah i am here and there i'm on a break right now but i plan on going back in the uh next semester uh probably to open up my own business no um no specific reason i just don't travel a lot i'm pretty local once a year can you be a little bit more specific no answer i like reading books i enjoy i enjoy cooking um exercising is great i'm i'm i'm pretty good at it um yeah um probably about two weeks ago uh frustrated um i don't like bias um i don't like um when someone says they're gonna do something and they don't uh somewhat friendship i like to play sports i enjoy uh going out with friends and family playing games grandparents parents um yeah i mean they've always given me great advice they've always kept it real real close i would say going to college right after high 

### Function to combine all text files + patient id to one df

In [ ]:
def combine_transcripts(directory, label_filepath):
    column_names_final = ["Participant_ID", "Transcript", "PHQ8_Binary"]
    df_final = pd.DataFrame(columns = column_names_final)
    df_labels = pd.read_csv(label_filepath)

    csv_files = [pos_csv for pos_csv in os.listdir(directory) if pos_csv.endswith('_P')]

    csv=[]
    for i in range(len(csv_files)):
      csv.append([n for n in os.listdir(transcript_directory  + csv_files[i]) if n.endswith('TRANSCRIPT.csv')])
    print(len(csv))
  # to fix the problem of 2d arrays in csv
    csv_l=[]
    for i in range(len(csv)-1):
      for k in csv[i]:
        csv_l.append(k)
    print("CSV_l")
    print(len(csv_l))
    print(csv_l[-1])
  # to reduce the data to the train size
    final_csv=[]
    numbers=df_labels["Participant_ID"]
    for k in csv_l:
      for i in numbers:
        if k[0:3] == str(i):
          final_csv.append(k)
    print("Final Fianl")    
    print(len(final_csv))




    for filename in final_csv:
      filepath = directory  + (filename[0:3] + "_P/")+ filename
      participant_no, transcript_concat = get_transcript(filepath)
      df_labels = pd.read_csv(label_filepath, index_col = "Participant_ID")
      PHQ_Binary = df_labels.loc[participant_no]["PHQ8_Binary"]
      # PHQ_Score = df_labels.loc[participant_no]["PHQ_Score"]

      new_row = {"Participant_ID": participant_no, "Transcript": transcript_concat, "PHQ8_Binary": PHQ_Binary}
      df_final = df_final.append(new_row, ignore_index = True)
      
    return df_final

In [ ]:
df1 = combine_transcripts(transcript_directory, label_filepath).sort_values(by=['Participant_ID'], ascending=True)
df1.to_csv("raw_compiled_transcripts.csv", index=False)
df1 = df.reset_index(drop=True)

df.head()

189
CSV_l
189
491_TRANSCRIPT.csv
Final Fianl
107


,Participant_ID,Transcript,PHQ8_Binary
0,303,okay how 'bout yourself here in california yea...,0.0
1,304,i'm doing good um from los angeles california ...,0.0
2,305,i'm doing alright uh originally i'm from calif...,0.0
3,310,yes it's okay <laughter> fine <laughter> i liv...,0.0
4,312,yes fine how about you here yes the weather we...,0.0


In [ ]:
df2 = combine_transcripts(transcript_directory, validation).sort_values(by=['Participant_ID'], ascending=True)
df2.to_csv("raw_compiled_transcripts.csv", index=False)
df2 = df2.reset_index(drop=True)

df2.head()

189
CSV_l
189
491_TRANSCRIPT.csv
Final Fianl
34
error
error


,Participant_ID,Transcript,PHQ8_Binary
0,302,i'm fine how about yourself i'm from los ange...,0
1,307,<laughter> um moscow um my family moved to the...,0
2,331,yes okay connecticut um to be an actor <lau...,0
3,335,yes i'm okay uh i'm from here originally los ...,1
4,346,yes i'm okay here in los angeles there's a ...,1


In [ ]:
df2.isnull().values.any()

False

In [ ]:
df = pd.concat([df1, df2], axis=0, ignore_index=True)


In [ ]:
df.Transcript[33]

"yes  i'm doing pretty good <laughter>  i'm from oklahoma  mhm yup oklahoma city  uh the weather is great i love being near the ocean and the palm trees and uh mountains i just love it here <laughter> um it was pretty easy there's a few differences it's a huge city compared to where i'm from so traffic has definitely been an adjustment for me <laughter>  um i just always wanted to live in california uh hated being landlocked in oklahoma just wanted to get out to the west coast be near the ocean   i studied psychology and minored in art  um no not at the moment  um well actually i think i misunderstood your question i'm not still working on my degree but i am working in the field of psychology i well i just got hired working at a group home for teenage boys who have been in trouble with the law  my dream job oh  i don't really know i think i just wanna help people however i can and um i just like working with people in the realm of psychology so i think i may already have my dream job <

In [ ]:
df["PHQ8_Binary"].value_counts()

0.0    99
1.0    42
Name: PHQ8_Binary, dtype: int64

### Upsampling technique

In [ ]:
# DID NOT WORK
# !pip install paraphraser

# from paraphraser import paraphrase

In [ ]:
df.head()

,Participant_ID,Transcript,PHQ8_Binary
0,303,okay how 'bout yourself here in california yea...,0.0
1,304,i'm doing good um from los angeles california ...,0.0
2,305,i'm doing alright uh originally i'm from calif...,0.0
3,310,yes it's okay <laughter> fine <laughter> i liv...,0.0
4,312,yes fine how about you here yes the weather we...,0.0


In [ ]:
TARGET_COUNT = 80

df_neg = df[df.PHQ8_Binary == 0]
df_pos = df[df.PHQ8_Binary == 1]

ori_neg_ids = df_neg.Participant_ID.values
ori_pos_ids = df_pos.Participant_ID.values

print(f"No of non-depressed: {len(ori_neg_ids)}. No of depressed: {len(ori_pos_ids)}")

No of non-depressed: 77. No of depressed: 30


In [ ]:
def get_sampling_frame(ls, target_count):
    """
    Returns the sampling frame to be paraphrased, excluding the original sampling frame
    """
    random.seed(42)
    
    new_ls = []
    quotient = int(target_count / len(ls))
    remainder = target_count % len(ls)
    
    # if target_count >= size of given list
    if quotient > 0:
        # only fair to get every element
        new_ls = ls * (quotient - 1)
    
    # fill up the remaining spots
    new_ls += random.sample(ls, remainder)
    
    return new_ls

In [ ]:
# negative sampling frame
import random

aug_neg_ids = sorted(get_sampling_frame(list(ori_neg_ids), TARGET_COUNT))

print(aug_neg_ids)

[310, 328, 391]


In [ ]:
# positive sampling frame
aug_pos_ids = sorted(get_sampling_frame(list(ori_pos_ids), TARGET_COUNT))

print(aug_pos_ids)

[319, 319, 320, 320, 321, 321, 325, 325, 330, 330, 338, 339, 344, 344, 345, 345, 347, 347, 348, 350, 351, 352, 352, 353, 355, 356, 356, 362, 362, 372, 372, 376, 380, 380, 386, 386, 402, 412, 412, 414, 414, 426, 426, 433, 441, 441, 448, 448, 459, 459]


### Parrot Paraphrasing tool

In [ ]:
! pip install git+https://github.com/PrithivirajDamodaran/Parrot.git


  Cloning https://github.com/PrithivirajDamodaran/Parrot.git to /tmp/pip-req-build-ngq14cvk
  Running command git clone -q https://github.com/PrithivirajDamodaran/Parrot.git /tmp/pip-req-build-ngq14cvk
     |████████████████████████████████| 3.4 MB 5.1 MB/s 
     |████████████████████████████████| 1.2 MB 35.0 MB/s 
     |████████████████████████████████| 50 kB 6.1 MB/s 
     |████████████████████████████████| 78 kB 6.1 MB/s 
     |████████████████████████████████| 6.8 MB 51.9 MB/s 
     |████████████████████████████████| 61 kB 475 kB/s 
     |████████████████████████████████| 596 kB 59.2 MB/s 
     |████████████████████████████████| 3.3 MB 47.7 MB/s 
     |████████████████████████████████| 895 kB 61.3 MB/s 
  Created wheel for parrot: filename=parrot-1.0-py3-none-any.whl size=8695 sha256=cfd9eba75c12355fe095a398e8ec4688f381add675e602abfa4bf5e0e6ac03fa
  Stored in directory: /tmp/pip-ephem-wheel-cache-v0qjpact/wheels/d8/27/d8/b757755a5b0b804c59714fdd8a4e2733c20802f638f17f7d96
  Created 

In [ ]:
from parrot import Parrot
import torch
import warnings
warnings.filterwarnings("ignore")

In [ ]:
def random_state(seed):
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

random_state(1234)

In [ ]:
parrot = Parrot(model_tag="prithivida/parrot_paraphraser_on_T5", use_gpu=False)

Downloading:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.34k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.74k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
phrases = ["What's the most delicious papayas?"]


In [ ]:
for phrase in phrases:
  print("-"*100)
  print("Input_phrase: ", phrase)
  print("-"*100)
  para_phrases = parrot.augment(input_phrase=phrase)
  # for para_phrase in para_phrases:
  print(para_phrases)

----------------------------------------------------------------------------------------------------
Input_phrase:  What's the most delicious papayas?
----------------------------------------------------------------------------------------------------
[('tell me the best tasting papaya you have ever had?', 43), ('tell me the best papaya recipe?', 35), ('recommend some best recipes for papaya?', 30), ('show some of the best papayas?', 28), ('can you list the best papayas?', 27), ('which are the best papayas?', 27), ("what's the best papaya?", 25), ('tell me the most delicious papaya?', 18), ('list some of the most delicious papayas?', 16)]


**THE END**

In [ ]:
# Before paraphrasing
before_text = ["i just want to paraphrase this sentence"]

before_text

['i just want to paraphrase this sentence']

In [ ]:
# After paraphrasing
after_text = parrot.augment(before_text[0])

after_text[1][0]

'i just want to just make a sentence'

In [ ]:
aug_neg_ids[0]

310

In [ ]:
df.head()

,Participant_ID,Transcript,PHQ8_Binary
0,303,okay how 'bout yourself here in california yea...,0.0
1,304,i'm doing good um from los angeles california ...,0.0
2,305,i'm doing alright uh originally i'm from calif...,0.0
3,310,yes it's okay <laughter> fine <laughter> i liv...,0.0
4,312,yes fine how about you here yes the weather we...,0.0


In [ ]:
a = df[df.Participant_ID == 319]['Transcript'].values[0]
a


"sure mm okay los angeles um well <laughter> that's a good question um i like the familiarity with everything i know where everything is in the city mm that's all i can say right now so spread out so hard to get places  is there anything else my dream job i don't think there is a dream job um for me no there isn't one mm i'm uh sort of unemployed right now <sigh> so am i <cough> mm that's true <sniffle> i started i studied urban planning well um i think this um school shooting thing and uh the n_r_a seeming to not wanna budge on this gun control issue thing that's it mm um i try to remove myself from that situation that's annoying me <sniffle>  i've gotten better over the years um most of the time i just try to um you know walk away or you know control my emotions in terms of you know just walking away mm just watch t_v now hmm i would say my children being born well i have four children um each birth was speical and um <sniffle> you know it was a unique experience each time well it's 

In [ ]:
b = df[df.Participant_ID == 319]['PHQ8_Binary'].values[0]
b

1.0

In [ ]:
para_text = parrot.augment(a)


NameError: ignored

In [ ]:
import time

def get_aug_df(ori_df, aug_ls_ids):
    aug_df = pd.DataFrame(columns=['Transcript', 'PHQ8_Binary'])
    
    # there is gtoken issue intermittently so paraphrasing will run into errors
    while len(aug_ls_ids) != 0:

        participant_id = aug_ls_ids[0]
        
        try: 
            ori_text = ori_df[ori_df.Participant_ID == participant_id]['Transcript'].values[0]
            label = ori_df[ori_df.Participant_ID == participant_id]['PHQ8_Binary'].values[0]
            para_text = parrot.augment(ori_text)
            if ori_text == para_text[1][0]: # sometimes will return you the same text back
                continue
            else:
                aug_df = aug_df.append({'Transcript': para_text, 'PHQ8_Binary': label}, ignore_index=True)
                aug_ls_ids.remove(participant_id)
            
        except Exception:
            time.sleep(2)
    
    return aug_df

# aug_df = get_aug_df(df_train, aug_pos_ids + aug_neg_ids)

# frames = [aug_df, df_train.drop(columns=['PHQ8_Binary'])]
# pd.concat(frames)

In [ ]:
aug_df = get_aug_df(df, aug_pos_ids + aug_neg_ids)

frames = [aug_df, df_train.drop(columns=['PHQ8_Binary'])]
pd.concat(frames)

KeyboardInterrupt: ignored

In [ ]:


# before
df[df.Participant_ID == 338]['Transcript'].values[0]

"yes  i'm doing fine  i'm from los angeles california  wow wonderful weather um outdoor activities all the time too crowded pollution  the norm  i studied child communications um recently i just i would say yes preschool teacher  well i've always worked with children ever since i was eighteen years old so i figured doing something helping someone it doesn't pay well but you're investing in the future  wow some job that i could get paid to be outside all day get paid well to be outside all day  yes  i'm kinda quiet but i also like i probably would say borderline i like outside activities but i mainly kinda do um activities probably by myself so yeah i would say i'm an introvert slightly uh wow just um maybe a racist or disrespectful person i kinda shut down and and reevaluate how i am as a person how i'm coming off to other people i can't change anyone else's personality but i can work on my own   i'm pretty good  i usually don't argue with people i usually walk away i mean a argument d

In [ ]:
# after
df[df.Participant_ID == 338]['Transcript'].values[1]

In [ ]:
augmented_transcript_file = 'augmented_compiled_transcripts.csv'

aug_df = df

# before
aug_df[aug_df.Participant_ID == 338]['Transcript'].values[0]

"yes  i'm doing fine  i'm from los angeles california  wow wonderful weather um outdoor activities all the time too crowded pollution  the norm  i studied child communications um recently i just i would say yes preschool teacher  well i've always worked with children ever since i was eighteen years old so i figured doing something helping someone it doesn't pay well but you're investing in the future  wow some job that i could get paid to be outside all day get paid well to be outside all day  yes  i'm kinda quiet but i also like i probably would say borderline i like outside activities but i mainly kinda do um activities probably by myself so yeah i would say i'm an introvert slightly uh wow just um maybe a racist or disrespectful person i kinda shut down and and reevaluate how i am as a person how i'm coming off to other people i can't change anyone else's personality but i can work on my own   i'm pretty good  i usually don't argue with people i usually walk away i mean a argument d

In [ ]:
# after
aug_df[aug_df.Participant_ID == 338]['Transcript'].values[1]

IndexError: ignored

### Embedding

In [ ]:
!pip install contractions


In [ ]:
from nltk.corpus import stopwords

nltk.download('stopwords')
stop_words = list(stopwords.words('english'))

# # depressed persons tend to use first-person pronouns more, and third-person pronouns less. these words might provide indication
# excluded_pronouns = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll",
#                     "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's",
#                      'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves']

# for pronoun in excluded_pronouns:
#     stop_words.remove(pronoun)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
df.head()

,Participant_ID,Transcript,PHQ8_Binary
0,303,okay how 'bout yourself here in california yea...,0
1,304,i'm doing good um from los angeles california ...,0
2,305,i'm doing alright uh originally i'm from calif...,0
3,310,yes it's okay <laughter> fine <laughter> i liv...,0
4,312,yes fine how about you here yes the weather we...,0


In [ ]:
from nltk.tokenize import TweetTokenizer
import contractions
import re as re
import string

def filter_text(text):
    
    text = re.sub('<[^<]+?>', '', text) # remove anything enclosed by tags e.g. <>
    text = re.sub('\[(.*?)\]', '', text) # remove anything enclosed by closed brackets i.e. []
    text = contractions.fix(text) # expands contractions e.g.'he's happy' -> 'he is happy'
    
    text = text.lower() # contractions will capitalize 'i'
    text = text.translate(str.maketrans("", "", string.punctuation)) # remove remaining punctuations e.g. apostrophe
    
    tokens = TweetTokenizer().tokenize(text) # use TweetTokenizer as transcripts contain informal texts
    
    filtered_sentence = [w for w in tokens if w not in stop_words]
    text = ' '.join(filtered_sentence)
    
    return text

In [ ]:
original_text = df.Transcript[1]

df.Transcript = df.Transcript.apply(filter_text)
filtered_text = df.Transcript[1]

print(f"The original text is:\n{original_text}")
print("-------------------------------------------")
print(f"The filtered text is:\n{filtered_text}")

The original text is:
i'm doing good um from los angeles california um the cool weather the beaches the people the schools  um child development yes to be a teacher for special education it's not hard 'cause i love to work with children yes dishonesty maybe um like if a guy like especially when it comes to relationships when they lie to you about something that's very minimal my mom i wanna say my mom um very close we talk every day very close even though i don't live with them i try to see them as much as i can uh it's okay <laughter> it's it's good it's yes i do  um they're they're friendly it's just that they're very quiet 'cause i'm not used to that environment it's fine like the how much it costs is very cheap so that's good and they're friendly so it's fine I just wish it was a little bit it could've been better so yeah  yes <laughter>  um i had a different option to live with somebody else and i'm still friends with the female and comes to find out that we connect more than the 

In [ ]:
df.to_csv("/content/drive/MyDrive/DAIC-WOZ/clean_compiled_transcripts.csv", index=False)
df.head()

,Participant_ID,Transcript,PHQ8_Binary
0,303,okay bout california yeah oh well big broad lo...,0
1,304,good um los angeles california um cool weather...,0
2,305,alright uh originally california uh born glend...,0
3,310,yes okay fine live hollywood uh seattle uh kin...,0
4,312,yes fine yes weather weather mainly yeah main ...,0


In [ ]:
ps=nltk.PorterStemmer()

df.Transcript = df.Transcript.apply(lambda x: " ".join(ps.stem(term) for term in x.split()))


In [ ]:
df.Transcript[0]

'okay bout california yeah oh well big broad lot lot um um job opportun state um pretti much big lot traffic um mayb violenc rate bad news even though know want know go environ still watch know look see go daili basi sociolog um two year degre liber art major sociolog actual mta bu oper well sinc metro steadi grow um dream job move compani actual start class supervisori next week um look promot within compani good thing lot opportun metro take class pay tuition thing natur thank well actual hard right easi take advantag right look promot within know sinc alreadi work good thing good go ahead take class need interest go ahead move ladder soon possibl mean sorri okay read take long walk hot bath medit close eye sometim pretti much good see bu oper run circumst situat got remain calm still remain profession time well look like goe mean come experi know um someth continu basi usual somewher line becom good know look like drive year deal public handl circumst know treat peopl like want trea

In [ ]:
df1.Transcript[0]

"okay how 'bout yourself here in california yeah oh well that it's big and broad there's a lot to do a lot of um um job opportunities than other states um pretty much that it's big and there's a lot you can do here traffic um maybe the violence rate bad news even though you know you wanna know what's going on in your environment but you still have to watch it so you can you know look out to see what's going on on a daily basis sociology i have a um two year degree in liberal arts but my major was sociology no i'm actually an m_t_a bus operator well since metro is steady growing um my dream job is to move up in the company and i'm about to actually start some classes for supervisory next week so um 'cause they look to promote within the company so that's the good thing there's a lot of opportunities at metro where you can take classes and they'll pay for your tuition and things of that nature thanks well it's not actually hard as right now it's easy if you take advantage of it right now

In [ ]:
df.Transcript[0][0][0]

'o'

### Feature Engineering

**Текст, выделенный полужирным шрифтом**

In [ ]:
from nltk.tokenize import word_tokenize
nltk.download('punkt')

# creating a bag-of-words

all_words=[]

for message in df.Transcript:
  words=word_tokenize(message)
  for w in words:
    all_words.append(w)


all_words=nltk.FreqDist(all_words)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# printing the total num of words and 15 most common

print("Number of words: {}".format(len(all_words)))

print("Most common words: {}".format(all_words.most_common(15)))

Number of words: 4689
Most common words: [('um', 5214), ('uh', 3747), ('like', 2655), ('know', 2649), ('realli', 1349), ('go', 1123), ('would', 1039), ('thing', 1013), ('think', 1002), ('yeah', 946), ('get', 936), ('good', 861), ('well', 851), ('peopl', 850), ('time', 795)]


In [ ]:
# use 1500 most common words as features

word_features= list(all_words.keys())[:4000]

In [ ]:
word_features

['okay',
 'bout',
 'california',
 'yeah',
 'oh',
 'well',
 'big',
 'broad',
 'lot',
 'um',
 'job',
 'opportun',
 'state',
 'pretti',
 'much',
 'traffic',
 'mayb',
 'violenc',
 'rate',
 'bad',
 'news',
 'even',
 'though',
 'know',
 'want',
 'go',
 'environ',
 'still',
 'watch',
 'look',
 'see',
 'daili',
 'basi',
 'sociolog',
 'two',
 'year',
 'degre',
 'liber',
 'art',
 'major',
 'actual',
 'mta',
 'bu',
 'oper',
 'sinc',
 'metro',
 'steadi',
 'grow',
 'dream',
 'move',
 'compani',
 'start',
 'class',
 'supervisori',
 'next',
 'week',
 'promot',
 'within',
 'good',
 'thing',
 'take',
 'pay',
 'tuition',
 'natur',
 'thank',
 'hard',
 'right',
 'easi',
 'advantag',
 'alreadi',
 'work',
 'ahead',
 'need',
 'interest',
 'ladder',
 'soon',
 'possibl',
 'mean',
 'sorri',
 'read',
 'long',
 'walk',
 'hot',
 'bath',
 'medit',
 'close',
 'eye',
 'sometim',
 'run',
 'circumst',
 'situat',
 'got',
 'remain',
 'calm',
 'profession',
 'time',
 'like',
 'goe',
 'come',
 'experi',
 'someth',
 'contin

In [ ]:
#  define a find_features function

def find_feature(message):
  words= word_tokenize(message)
  features={}
  for word in word_features:
    features[word]=(word in words)
  
  return features

  

 
      

In [ ]:
df.Transcript[1]

'good um lo angel california um cool weather beach peopl school um child develop ye teacher special educ hard love work children ye dishonesti mayb um like guy like especi come relationship lie someth minim mom want say mom um close talk everi day close even though live tri see much uh okay good ye um friendli quiet use environ fine like much cost cheap good friendli fine wish littl bit could better yeah ye um differ option live somebodi els still friend femal come find connect roommat live kind regret move roommat versu suppos move begin stuck year whatev fine know yeah noth realli get realli mad think top head um um mayb come school treat children especi children color mayb realli get mad mm ye thing within guess um intern motiv thing guess yeah yeah wow hardli ever argu somebodi yeah oh sister guess sister month ago uh felt betray belittl littl sister disrespect type way adult yeah feel situat um get grown folk busi convers mother guess overheard said felt like action choos somebodi

In [ ]:
# Example
features= find_feature(df.Transcript[0])
for key, value in features.items():
  if value==True:
    print(key)
  
      

okay
bout
california
yeah
oh
well
big
broad
lot
um
job
opportun
state
pretti
much
traffic
mayb
violenc
rate
bad
news
even
though
know
want
go
environ
still
watch
look
see
daili
basi
sociolog
two
year
degre
liber
art
major
actual
mta
bu
oper
sinc
metro
steadi
grow
dream
move
compani
start
class
supervisori
next
week
promot
within
good
thing
take
pay
tuition
natur
thank
hard
right
easi
advantag
alreadi
work
ahead
need
interest
ladder
soon
possibl
mean
sorri
read
long
walk
hot
bath
medit
close
eye
sometim
run
circumst
situat
got
remain
calm
profession
time
like
goe
come
experi
someth
continu
usual
somewher
line
becom
drive
deal
public
handl
treat
peopl
younger
get
older
wiser
learn
differ
today
would
say
famili
kid
death
around
part
life
born
leav
one
day
keep
sane
love
set
exampl
first
teacher
posit
think
home
wake
whatev
taught
lookin
everi
whether
school
tendenc
tri
teach
moral
valu
along
regardless
race
four
half
old
son
seven
daughter
also
twenti
three
pregnant
boy
littl
girl
grandki

In [ ]:
processed=df.Transcript
Y= df.PHQ8_Binary

In [ ]:
processed

0      okay bout california yeah oh well big broad lo...
1      good um lo angel california um cool weather be...
2      alright uh origin california uh born glendal h...
3      ye okay fine live hollywood uh seattl uh kind ...
4      ye fine ye weather weather mainli yeah main we...
                             ...                        
136    ye fine born texa move california three grew c...
137    sure mm good guess new orlean louisiana uh two...
138    ye well born oakland um differ area citi uh di...
139    ye well thank san lui obispo ye four year ago ...
140    yeah alreadi northern illinoi two half year ag...
Name: Transcript, Length: 141, dtype: object

### Classification Report!!!!!

In [ ]:
df.Transcript[0]

0    okay how 'bout yourself here in california yea...
0    i'm fine how about yourself i'm from lo angel ...
Name: Transcript, dtype: object

In [ ]:
# Find features for all messages

messages= zip(processed, Y)

# define a seed for reproducibility
seed = 1

np.random.seed= seed
np.random.shuffle(df.Transcript)

# call find_feature function for all texts
featuresetes=[(find_feature(text), label) for (text, label) in messages]




In [ ]:
featuresetes[2][1]

In [ ]:
# split train and test 
from sklearn import model_selection

training, testing = model_selection.train_test_split(featuresetes, test_size=0.1, random_state = seed, stratify=Y)


In [ ]:
print(len(training))
print(len(testing))


### Scikit-Learn Classifier with NLTK

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix



In [ ]:
# Define models to train

names= [
         'SVM linear', 'K Nearest Neighbors', 'Decision Tree', 'Random Forest', 'Logistic Regression', 
        'SGD Classifier']
      

classifier = [
              SVC(kernel='poly', class_weight='balanced'),
              KNeighborsClassifier(),
              DecisionTreeClassifier(),
              RandomForestClassifier(),
              LogisticRegression(),
              SGDClassifier(max_iter=100, class_weight='balanced')
              
              
]

models= zip(names, classifier)

models

In [ ]:
from nltk.classify.scikitlearn import SklearnClassifier


In [ ]:
# wrap models in nltk
from nltk.classify.scikitlearn import SklearnClassifier

for name, model in models:
  nltk_model= SklearnClassifier(model)
  nltk_model.train(training)
  accuracyTrain= nltk.classify.accuracy(nltk_model, training) * 100
  accuracyVal= nltk.classify.accuracy(nltk_model, testing) * 100

  print('{}: Accuracy: {}'.format(name, accuracyTrain))
  print('{}: AccuracyVal: {}'.format(name, accuracyVal))

SVM linear: Accuracy: 98.4126984126984
SVM linear: AccuracyVal: 73.33333333333333
K Nearest Neighbors: Accuracy: 73.01587301587301
K Nearest Neighbors: AccuracyVal: 73.33333333333333
Decision Tree: Accuracy: 100.0
Decision Tree: AccuracyVal: 80.0
Random Forest: Accuracy: 100.0
Random Forest: AccuracyVal: 73.33333333333333
Logistic Regression: Accuracy: 100.0
Logistic Regression: AccuracyVal: 73.33333333333333
SGD Classifier: Accuracy: 100.0
SGD Classifier: AccuracyVal: 60.0


In [ ]:
# ensemble method - Voting classifier
from sklearn.ensemble import VotingClassifier

# Define models to train

names= ['SVM linear',
         'K Nearest Neighbors', 'Decision Tree', 'Random Forest', 'Logistic Regression', 'SGD Classifier', 'Naive Bayes'
         ]

classifier = [SVC(kernel='poly', class_weight='balanced'),
              KNeighborsClassifier(),
              DecisionTreeClassifier(),
              RandomForestClassifier(class_weight='balanced'),
              LogisticRegression(),
              SGDClassifier(max_iter=100, class_weight='balanced')
              
              
]

models= list(zip(names, classifier))


nltk_ensemble = SklearnClassifier(VotingClassifier(estimators = models, voting = 'hard', n_jobs= -1))

nltk_ensemble.train(training)
accuracyTr= nltk.classify.accuracy(nltk_ensemble, training) * 100
accuracyVa= nltk.classify.accuracy(nltk_ensemble, testing) * 100
print('{} Ensemble Method Accuracy: {}'.format(name, accuracyTr))

print('{} Ensemble Method Accuracy: {}'.format(name, accuracyVa))


SGD Classifier Ensemble Method Accuracy: 100.0
SGD Classifier Ensemble Method Accuracy: 73.33333333333333


### Prediction on test

In [ ]:
# make class label predictions for testing set
txt_features, labels=zip(*testing)

prediction= nltk_model.classify_many(txt_features)


In [ ]:
# print a confusionn matrix n classification report
print(classification_report(labels, prediction))

pd.DataFrame(
    confusion_matrix(labels, prediction),
    index=[['actual', 'actual'], ['not depressed', 'depressed']],
    columns=[['predicted', 'predicted'], ['not depressed', 'depressed']]
)


              precision    recall  f1-score   support

         0.0       0.73      0.73      0.73        11
         1.0       0.25      0.25      0.25         4

    accuracy                           0.60        15
   macro avg       0.49      0.49      0.49        15
weighted avg       0.60      0.60      0.60        15



predicted          
                     not depressed depressed
actual not depressed             8         3
       depressed                 3         1

In [ ]:
# make class label predictions for testing set
txt_features, labels=zip(*testing)

prediction= nltk_ensemble.classify_many(txt_features)

# print a confusionn matrix n classification report
print(classification_report(labels, prediction))

pd.DataFrame(
    confusion_matrix(labels, prediction),
    index=[['actual', 'actual'], ['not depressed', 'depressed']],
    columns=[['predicted', 'predicted'], ['not depressed', 'depressed']]
)


              precision    recall  f1-score   support

         0.0       0.73      1.00      0.85        11
         1.0       0.00      0.00      0.00         4

    accuracy                           0.73        15
   macro avg       0.37      0.50      0.42        15
weighted avg       0.54      0.73      0.62        15



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


predicted          
                     not depressed depressed
actual not depressed            11         0
       depressed                 4         0

### GridSearch


SVM

In [ ]:
from sklearn.model_selection import GridSearchCV
 
# defining parameter range
param_grid = {'C': [0.1, 1, 10, 100, 1000],
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf', 'poly', 'linear']}
 
grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3)
 
# fitting the model for grid search
# grid.fit(training)
modelWithBestParam= SklearnClassifier(grid)
modelWithBestParam.train(training)

In [ ]:
print(grid.best_params_)
 
# print how our model looks after hyper-parameter tuning
print(grid.best_estimator_)

{'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}
SVC(C=100, gamma=0.001)


In [ ]:
from yellowbrick.classifier import ClassificationReport
m=SVC(C= 100, gamma= 0.001, kernel= 'rbf', class_weight='balanced')
mClassReport= SklearnClassifier(m)
classes=['depressed', 'undepressed']
visualizer = ClassificationReport(mClassReport,  support=True)

txt, labels_train=zip(*training)

visualizer.fit(txt, labels_train)        # Fit the visualizer and the model

txt_features, labels=zip(*testing)

# prediction= nltk_model.classify_many(txt_features)

visualizer.score(txt_features, labels)        # Evaluate the model on the test data
visualizer.show()                       # Finalize and show the figure

YellowbrickTypeError: ignored

In [ ]:
# make class label predictions for testing set
txt_features, labels=zip(*testing)

prediction= modelWithBestParam.classify_many(txt_features)

# print a confusionn matrix n classification report
print(classification_report(labels, prediction))

pd.DataFrame(
    confusion_matrix(labels, prediction),
    index=[['actual', 'actual'], ['not depressed', 'depressed']],
    columns=[['predicted', 'predicted'], ['not depressed', 'depressed']]
)


              precision    recall  f1-score   support

         0.0       0.73      1.00      0.85        11
         1.0       0.00      0.00      0.00         4

    accuracy                           0.73        15
   macro avg       0.37      0.50      0.42        15
weighted avg       0.54      0.73      0.62        15



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


predicted          
                     not depressed depressed
actual not depressed            11         0
       depressed                 4         0

#### RandomForest

In [ ]:
from sklearn.model_selection import GridSearchCV
 
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}
# Create a based model
rf = RandomForestClassifier()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)


modelGridRF= SklearnClassifier(grid_search)
modelGridRF.train(training)




Fitting 3 folds for each of 288 candidates, totalling 864 fits


<SklearnClassifier(GridSearchCV(cv=3, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'bootstrap': [True], 'max_depth': [80, 90, 100, 110],
                         'max_features': [2, 3], 'min_samples_leaf': [3, 4, 5],
                         'min_samples_split': [8, 10, 12],
                         'n_estimators': [100, 200, 300, 1000]},
             verbose=2))>

In [ ]:
print(grid_search.best_params_)
 
# print how our model looks after hyper-parameter tuning
print(grid_search.best_estimator_)

print('-'*100)


# make class label predictions for testing set
txt_features, labels=zip(*testing)

prediction= modelGridRF.classify_many(txt_features)

# print a confusionn matrix n classification report
print(classification_report(labels, prediction))

pd.DataFrame(
    confusion_matrix(labels, prediction),
    index=[['actual', 'actual'], ['not depressed', 'depressed']],
    columns=[['predicted', 'predicted'], ['not depressed', 'depressed']]
)


{'bootstrap': True, 'max_depth': 80, 'max_features': 2, 'min_samples_leaf': 3, 'min_samples_split': 8, 'n_estimators': 100}
RandomForestClassifier(max_depth=80, max_features=2, min_samples_leaf=3,
                       min_samples_split=8)
----------------------------------------------------------------------------------------------------
              precision    recall  f1-score   support

         0.0       0.73      1.00      0.85        11
         1.0       0.00      0.00      0.00         4

    accuracy                           0.73        15
   macro avg       0.37      0.50      0.42        15
weighted avg       0.54      0.73      0.62        15



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


predicted          
                     not depressed depressed
actual not depressed            11         0
       depressed                 4         0